<a href="https://colab.research.google.com/github/IanMerlini/licao/blob/main/ComparalelaOPENACC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile timer.h

#include <stdio.h>
#include <math.h>
#include <sys/time.h>

// Funções para medição de tempo
struct timeval timerStart;

void StartTimer() {
    gettimeofday(&timerStart, NULL);
}

double GetTimer() {
    struct timeval timerStop, timerElapsed;
    gettimeofday(&timerStop, NULL);
    timersub(&timerStop, &timerStart, &timerElapsed);
    return timerElapsed.tv_sec * 1000.0 + timerElapsed.tv_usec / 1000.0;
}

// Função que será integrada
double f(double x) {
    return 4.0 / (1.0 + x * x);
}

int main() {
    const int n = 100000000; // Número de trapézios
    const double a = 0.0, b = 1.0; // Limites de integração
    double h = (b - a) / n; // Altura de cada trapézio
    double integral = 0.0;

    printf("Calculando a integral usando OpenACC...\n");

    StartTimer();

    #pragma acc parallel loop reduction(+:integral)
    for (int i = 0; i < n; i++) {
        double x_i = a + i * h;
        double x_next = a + (i + 1) * h;
        integral += (f(x_i) + f(x_next)) * h / 2.0;
    }

    double runtime = GetTimer();

    printf("Resultado da integral: %.12f\n", integral);
    printf("Tempo de execução: %.3f ms\n", runtime);

    return 0;
}

Writing timer.h


In [3]:
%%writefile task2_solution.c

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

// Funções para medição de tempo
struct timeval timerStart;

void StartTimer() {
    gettimeofday(&timerStart, NULL);
}

double GetTimer() {
    struct timeval timerStop, timerElapsed;
    gettimeofday(&timerStop, NULL);
    timersub(&timerStop, &timerStart, &timerElapsed);
    return timerElapsed.tv_sec * 1000.0 + timerElapsed.tv_usec / 1000.0;
}

#define N 1024 // Tamanho da matriz

void matrix_multiplication(float A[N][N], float B[N][N], float C[N][N]) {
    #pragma acc parallel loop collapse(2)
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            float sum = 0.0;
            for (int k = 0; k < N; k++) {
                sum += A[i][k] * B[k][j];
            }
            C[i][j] = sum;
        }
    }
}

int main() {
    float (*A)[N] = malloc(sizeof(float[N][N]));
    float (*B)[N] = malloc(sizeof(float[N][N]));
    float (*C)[N] = malloc(sizeof(float[N][N]));

    // Inicializando as matrizes
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            A[i][j] = rand() % 100;
            B[i][j] = rand() % 100;
            C[i][j] = 0.0;
        }
    }

    printf("Calculando a multiplicação de matrizes usando OpenACC...\n");

    StartTimer();

    matrix_multiplication(A, B, C);

    double runtime = GetTimer();

    printf("Multiplicação de matrizes concluída!\n");
    printf("Tempo de execução: %.3f ms\n", runtime);

    free(A);
    free(B);
    free(C);

    return 0;
}


Writing task2_solution.c


In [ ]:
!ls -l

total 12
drwxr-xr-x 1 root root 4096 Nov 19 14:23 sample_data
-rw-r--r-- 1 root root 1450 Nov 20 23:51 task2_solution.c
-rw-r--r-- 1 root root 1573 Nov 20 23:51 timer.h


In [ ]:
!wget https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-20-9_20.9_amd64.deb

--2024-11-20 23:51:08--  https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-20-9_20.9_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2117119120 (2.0G) [application/x-deb]
Saving to: ‘nvhpc-20-9_20.9_amd64.deb’

nvhpc-20-9_20.9_amd 100%[===================>]   1.97G   138MB/s    in 11s     

2024-11-20 23:51:19 (191 MB/s) - ‘nvhpc-20-9_20.9_amd64.deb’ saved [2117119120/2117119120]



In [ ]:
!wget https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-2020_20.9_amd64.deb

--2024-11-20 23:51:27--  https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-2020_20.9_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1272 (1.2K) [application/x-deb]
Saving to: ‘nvhpc-2020_20.9_amd64.deb’

nvhpc-2020_20.9_amd 100%[===================>]   1.24K  --.-KB/s    in 0s      

2024-11-20 23:51:27 (118 MB/s) - ‘nvhpc-2020_20.9_amd64.deb’ saved [1272/1272]



In [ ]:
!wget https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-20-9-cuda-multi_20.9_amd64.deb

--2024-11-20 23:51:29--  https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-20-9-cuda-multi_20.9_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1558347920 (1.5G) [application/x-deb]
Saving to: ‘nvhpc-20-9-cuda-multi_20.9_amd64.deb’

nvhpc-20-9-cuda-mul 100%[===================>]   1.45G   263MB/s    in 5.8s    

2024-11-20 23:51:35 (254 MB/s) - ‘nvhpc-20-9-cuda-multi_20.9_amd64.deb’ saved [1558347920/1558347920]



In [ ]:
!sudo apt-get install ./nvhpc-20-9_20.9_amd64.deb ./nvhpc-2020_20.9_amd64.deb ./nvhpc-20-9-cuda-multi_20.9_amd64.deb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvhpc-20-9' instead of './nvhpc-20-9_20.9_amd64.deb'
Note, selecting 'nvhpc-2020' instead of './nvhpc-2020_20.9_amd64.deb'
Note, selecting 'nvhpc-20-9-cuda-multi' instead of './nvhpc-20-9-cuda-multi_20.9_amd64.deb'
The following NEW packages will be installed:
  nvhpc-20-9 nvhpc-20-9-cuda-multi nvhpc-2020
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 0 B/3,675 MB of archives.
After this operation, 10.1 GB of additional disk space will be used.
Get:1 /content/nvhpc-2020_20.9_amd64.deb nvhpc-2020 amd64 20.9 [1,272 B]
Get:2 /content/nvhpc-20-9_20.9_amd64.deb nvhpc-20-9 amd64 20.9 [2,117 MB]
Get:3 /content/nvhpc-20-9-cuda-multi_20.9_amd64.deb nvhpc-20-9-cuda-multi amd64 20.9 [1,558 MB]
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5

In [ ]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/20.9/compilers/bin/nvc --version


nvc 20.9-0 LLVM 64-bit target on x86-64 Linux -tp haswell 
NVIDIA Compilers and Tools
Copyright (c) 2020, NVIDIA CORPORATION.  All rights reserved.


In [4]:
!nvidia-smi

Thu Nov 21 02:12:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/20.9/compilers/bin/nvc -acc -Minfo -gpu=cuda10.1 -fast -D__STRICT_ANSI__ -o task2_out task2_solution.c

/bin/bash: line 1: /opt/nvidia/hpc_sdk/Linux_x86_64/20.9/compilers/bin/nvc: No such file or directory


In [9]:
!gcc -o task2_out task2_solution.c -lm


In [10]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/22.11/compilers/bin/nvc -acc -Minfo -gpu=cuda -fast -o task2_out task2_solution.c


/bin/bash: line 1: /opt/nvidia/hpc_sdk/Linux_x86_64/22.11/compilers/bin/nvc: No such file or directory


In [11]:
!./task2_out

Calculando a multiplicação de matrizes usando OpenACC...
Multiplicação de matrizes concluída!
Tempo de execução: 6629.222 ms
